# Rotor Model



This Jupyter Notebook documents the workflow for building and evaluating a machine learning model to predict the thrust and torque coefficients of a rotor. The notebook includes the following steps:

1. **Import Libraries**: Import necessary libraries and modules for data processing, model building, and evaluation.
2. **Define Dataset Class**: Create a custom dataset class to load and preprocess the rotor data from CSV files.
3. **Extract and Organize Data**: Load the data, apply necessary transformations, and visualize the dataset.
4. **Define Neural Network Model**: Define an LSTM-based neural network model to predict the thrust and torque coefficients.
5. **Train the Model**: Train the model using the training dataset and evaluate it on the validation dataset.
6. **Plot Losses**: Plot the training and validation losses to monitor the model's performance.
7. **Save the Model**: Save the trained model and scalers for future use.
8. **Test the Model**: Evaluate the model on a separate testing dataset and compute performance metrics such as MAPE, R² score, and relative L2 norm error.


In [ ]:
import os
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset
import torch.nn.utils as nn_utils
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.fftpack import fft, ifft

import matplotlib.pyplot as plt

import joblib

In [ ]:
SAVE_TRAINED_MODEL = False
PLOT_RESULTS = True

project_path = '/mnt/e/eVTOL_model/eVTOL-VehicleModel/'
data_path = project_path + 'data/rotor_data/'
model_path = project_path + 'trained_models/'

In [ ]:
# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import datetime
date = datetime.date.today()

## Dataset class

In [ ]:
       
# Condition function to filter subdirectories
def subdir_condition(subdir_name):
    """
    Condition: Only process subdirectories whose names start with 'propeller-example_dji'.
    Modify this function to apply a specific filtering logic.
    """
    return subdir_name.startswith('rotor_dataset')  # Change this condition as needed


class PropellerDataset(Dataset):
    # def __init__(self, root_dir, alpha, J, theta, yaw, tilt, subdir_condition=None):
    def __init__(self, root_dir, subdir_condition=None):
        """
        Args:
            root_dir (string): Root directory with subdirectories containing CSV files.
            alpha (float): Angle of attack.
            J (float): Advance ratio.
            theta (float): Pitch.
            yaw (float): Yaw.
            tilt (float): Tilt.
            subdir_condition (callable, optional): A function or condition to filter subdirectories by name.
        """
        self.root_dir = root_dir
        self.data = []
        self.targets = []
        self.time_data = []  # Store time data separately
        self.omega_data = []  # Store omega (RPM) separately
        self.ct_data = []
        self.cq_data = []
        self.fft_ct_r = []
        self.fft_ct_i = []
        self.fft_cq_r = []
        self.fft_cq_i = []
        self.subdir_condition = subdir_condition

        # Traverse the root directory to gather data
        self._load_data()

    def _load_data(self):
        """
        Helper function to read CSV files from each subdirectory and extract relevant columns.
        """
        # Iterate through each subdirectory in the root directory
        for subdir, _, files in os.walk(self.root_dir):
            subdir_name = os.path.basename(subdir)
            
            # Apply subdirectory name condition
            if self.subdir_condition and not self.subdir_condition(subdir_name):
                continue

            for file in files:
                if file.endswith("_convergence.csv"):
                    # Load the CSV file
                    csv_path = os.path.join(subdir, file)
                    df = pd.read_csv(csv_path)
                    
                    # Extract necessary columns for input features
                    time = df['T'].values  # Time
                    omega = df['RPM_1'].values  # RPM
                    J = df['J'].values
                    AOA = df['AOA'].values
                    v_inf = df['v_inf'].values
                    pitch = df['Pitch (blade)'].values
                    tilt = df['Tilt'].values
                    yaw = df['Yaw'].values
                    ref_angle = df['ref age (deg)'].values

                    # Store time and omega in separate lists for easy access
                    self.time_data.append(time)
                    self.omega_data.append(omega)
                    
                    # Extract CT and CQ as output variables
                    ct = df['CT_1'].values  # Thrust coefficient (CT)
                    cq = df['CQ_1'].values  # Torque coefficient (CQ)

                    fft_ct = fft(ct)
                    fft_ct_real = np.real(fft_ct)
                    fft_ct_imag = np.imag(fft_ct)

                    fft_cq = fft(cq)
                    fft_cq_real = np.real(fft_cq)
                    fft_cq_imag = np.imag(fft_cq)

                    
                    # Store ct and cq in separate lists for easy access
                    self.ct_data.append(ct)
                    self.cq_data.append(cq)
                    self.fft_ct_r.append(fft_ct_real)
                    self.fft_ct_i.append(fft_ct_imag)
                    self.fft_cq_r.append(fft_cq_real)
                    self.fft_cq_i.append(fft_cq_imag)

                    # For each simulation, the input sequence is structured as (n_timesteps, n_features)
                    sequence_inputs = []
                    sequence_outputs = []
                    for i in range(len(time)):
                        # Each time step has time, omega, and predefined variables: alpha, J, theta, yaw, tilt
                        input_data = [
                            time[i],  omega[i], AOA[i], v_inf[i], (10*np.sin(omega[i]*time[i])), 
                            (10*np.cos(omega[i]*time[i])), J[i], pitch[i], tilt[i], yaw[i]
                        ]
                        output_data = [ct[i], cq[i]]
                        
                        
                        sequence_inputs.append(input_data)
                        sequence_outputs.append(output_data)

                    sequence_inputs = np.array([sequence_inputs], dtype=float)
                    sequence_outputs = np.array([sequence_outputs], dtype=float)

                    # Append input sequence (n_timesteps, num_features) and output (CT, CQ)
                    self.data.append(sequence_inputs)
                    self.targets.append(sequence_outputs)  # Append the whole CT and CQ sequences


    def __len__(self):
        """
        Returns the total number of sequences in the dataset.
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Returns a single sequence and its targets.
        """
        print(f"Requested index: {idx}")
        print(f"Dataset length: {len(self.data)}")  # Check if idx exceeds this
        print(f"Targets length: {len(self.targets)}")
        
        inputs = self.data[idx]  # Input sequence: (n_timesteps, n_features)
        targets = self.targets[idx]  # Output: (n_timesteps, 2)
        
        # return inputs, targets
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)

    def get_variable(self, variable_name):
        """
        Returns a list of arrays for the specified variable.
        Args:
            'time' - timesteps
            'omega' - rotational velocity [RPM]
            'CT' - Thrust coefficient
            'CQ' - Torque coefficient
            'fft_ct_r' - Thrust ccoefficient FFT - real part
            'fft_ct_i' - Thrust ccoefficient FFT - imag part
            'fft_cq_r' - Torque ccoefficient FFT - real part
            'fft_cq_i' - Torque ccoefficient FFT - imag part

        """
        if variable_name == 'time':
            return self.time_data  # Return all time steps for each simulation
        elif variable_name == 'omega':
            return self.omega_data  # Return all omega (RPM) values for each simulation
        elif variable_name == 'CT':
            return self.ct_data 
        elif variable_name == 'CQ':
            return self.cq_data
        elif variable_name == 'fft_ct':
            return self.fft_ct_r, self.fft_ct_i 
        elif variable_name == 'fft_cq':
            return self.fft_cq_r, self.fft_cq_i  
        else:
            raise ValueError(f"Variable {variable_name} not supported.")



In [ ]:
# Define the custom dataset class
# Creates training and validation datasets
class SimulationDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx], dtype=torch.float32), torch.tensor(self.outputs[idx], dtype=torch.float32)

## Extract and organize data

In [ ]:
def visualize_tabular_dataset(inputs, targets, visualize:bool):
    
    if visualize:
        # Additional part to visualize the data
        df_ips = [None]*len(inputs)
        df_ops = [None]*len(targets)

        for num_batch in range(len(inputs)):
            df_ips[num_batch] = pd.DataFrame(inputs[num_batch], columns=["timestep", "sin_comp", "cos_comp", "Omega", "alpha", "J", "theta", "tilt", "yaw"])
            df_ops[num_batch] = pd.DataFrame(targets[num_batch], columns=["fft_ct_real", "fft_ct_imag", "fft_cq_real", "fft_cq_imag"])

        print("Input dataset\n", df_ips)
        print("Target dataset\n", df_ops)

In [ ]:
# Root directory where simulation subdirectories are stored
root_dir = data_path + "training_data/"

visualize_dataset = True         # Visualize loaded dataset
visualize_norm_dataset = False   # Visualize normalized dataset
visualize_nn = True              # Visulaize NN structure

In [ ]:
# Create the dataset with a subdirectory condition

dataset = PropellerDataset(root_dir, subdir_condition=subdir_condition)
inputs, outputs = dataset[0:]


input_tensor = inputs.squeeze(1)  # Reshaping
print("Input shape:", input_tensor.shape)   # Should print: torch.Size([Num_sumulations, tsteps_per_simulations, 10])

output_tensor = outputs.squeeze(1)
print("Output shape:",output_tensor.shape)  # Should print: torch.Size([Num_sumulations, tsteps_per_simulations, 4])


## NN Model

In [ ]:
# LSTM Model Definition
class LSTMNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout=0.1):
        super(LSTMNet, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, (hn, cn) = self.lstm(x)
        out = self.dropout(lstm_out)
        out = self.fc(lstm_out)
        return out
    

## Train the model

### Define Hyperparameters

In [ ]:
# Hyperparameters
input_size = 10             # Number of input features
hidden_size = 50            # Number of LSTM units (hidden state size)
output_size = 2             # Number of output features (e.g., 2 for thrust and torque coefficients)
num_layers = 4              # Number of LSTM layers
learning_rate = 5e-4         # Learning rate
num_epochs = 2500           # Number of training epochs
batch_size = 8              # Batch size 

### Normalize the dataset

In [ ]:
# Initialize scalers for input and output
input_scaler = MinMaxScaler()
output_scaler = MinMaxScaler()


# Reshape data for scaling (flatten along the time dimension)
inputs_reshaped = input_tensor.reshape(-1, input_size)
outputs_reshaped = output_tensor.reshape(-1, output_size)

# Fit and transform inputs and outputs
inputs_normalized = input_scaler.fit_transform(inputs_reshaped).reshape(input_tensor.shape)
outputs_normalized = output_scaler.fit_transform(outputs_reshaped).reshape(output_tensor.shape)

print("Normalized input shape:", inputs_normalized.shape)
print("Normalized output shape:", outputs_normalized.shape)


### Split the dataset

In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset randomly
inputs_train, inputs_val, outputs_train, outputs_val = train_test_split(
    inputs_normalized, outputs_normalized, test_size=0.35, random_state=42, shuffle=True
)

# Create datasets
train_dataset = SimulationDataset(inputs_train, outputs_train)
val_dataset = SimulationDataset(inputs_val, outputs_val)

# Create DataLoaders
trainDataLoader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  
valDataLoader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)  # No need to shuffle validation


### Training ...

In [ ]:
def custom_loss(pred, target):
    return torch.mean((pred - target) ** 2) + 0.3 * torch.mean(torch.abs(pred - target))

In [ ]:

# Instantiate model, loss function, and optimizer
model = LSTMNet(input_size, hidden_size, output_size, num_layers).to(device)    # LSTM model
# criterion = nn.MSELoss()                                                      # Loss function                                     
criterion = custom_loss   

optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Lists to store losses
train_losses = []
eval_losses = []

print("[INFO] training the network...")

# Training Loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, targets in trainDataLoader:
        inputs, targets = inputs.squeeze(1).to(device), targets.squeeze(1).to(device)

        # Forward pass
        outputs = model(inputs)
        
        loss = criterion(outputs, targets)
        # Backward pass and optimization
        optimizer.zero_grad()

        # Apply gradient clipping
        nn_utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Clipping

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    avg_train_loss = running_loss / len(trainDataLoader)
    train_losses.append(avg_train_loss)


    # Evaluation Loop
    model.eval()
    eval_loss = 0.0

    with torch.no_grad():
        for inputs, targets in valDataLoader:
            inputs, targets = inputs.squeeze(1).to(device), targets.squeeze(1).to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            eval_loss += loss.item()

        avg_eval_loss = eval_loss / len(valDataLoader)
        eval_losses.append(avg_eval_loss)
    
    print(f'Epoch [{epoch+1}/{num_epochs}],\n Training Loss: {running_loss/len(trainDataLoader):.8f}')
    print(f'Evaluation Loss: {eval_loss/len(valDataLoader):.8f}')

print("\nFinished Training...")

### Plot the losses

In [ ]:
plt.figure(figsize=(10, 6), dpi=300)  # High-resolution plot

plt.plot(train_losses, color='black', linewidth=1.5, label="Training loss")
plt.plot(eval_losses, color='red', linewidth=1.5, label="Validation loss")
plt.title("Training Loss and Validation Loss", fontsize=12)
plt.grid(True, linestyle='--', linewidth=0.5)
plt.xlabel("Epoch", fontsize=10)
plt.ylabel("MSE Loss", fontsize=10)
plt.legend(loc='upper right', fontsize=10)
plt.savefig("rotor_model_loss.pdf", format="pdf", dpi=300, bbox_inches="tight")
# plt.savefig('loss_curve.png')
# plt.show()

### Save the model

In [ ]:
# Save the model if needed
if SAVE_TRAINED_MODEL:

    model_save_path =  model_path + '/models/rotor/{}_scaled_H26FpropModel_MSELoss_lr{}_e{}_nL{}_numNN{}.pth'.format(date, learning_rate, num_epochs, num_layers, hidden_size)
    print("The model will be saved as the following:\n {}".format(model_save_path))
    torch.save(model.state_dict(), model_save_path)

    # save the scalers
    scaler_save_path = model_path + '/scalers/rotor/'
    joblib.dump(input_scaler, scaler_save_path+'/{}_scaled_H26F_MSELoss_ipScaler_lr{}_e{}_nL{}_numNN{}.pkl'.format(date, learning_rate, num_epochs, num_layers, hidden_size))
    joblib.dump(output_scaler, scaler_save_path+'/{}_scaled_H26F_MSELoss_opScaler_lr{}_e{}_nL{}_numNN{}.pkl'.format(date, learning_rate, num_epochs, num_layers, hidden_size))

## Test the model

In [ ]:
from sklearn.metrics import r2_score

# Root directory where simulation subdirectories are stored
root_dir_test_base = data_path + "testing_data/"

def mape(y_true, y_pred):
    """Compute Mean Absolute Percentage Error (MAPE)"""
    mask = y_true != 0  # Avoid division by zero
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def relative_l2_norm(y_true, y_pred):
    """Compute Relative L2 Norm Error (ε)"""
    mask = y_true != 0  # Avoid division by zero
    numerator = np.linalg.norm(y_pred[mask] - y_true[mask], ord=2)  # ||pred - true||_2
    denominator = np.linalg.norm(y_true[mask], ord=2)  # ||true||_2
    return (numerator / denominator) * 100

# Initialize lists to store evaluation results
mape_ct_list, mape_cq_list = [], []
r2_ct_list, r2_cq_list = [], []
relative_l2_norm_ct_list, relative_l2_norm_cq_list = [], []

for simulation_case in os.listdir(root_dir_test_base):
    # Root directory where simulation subdirectories are stored
    root_dir_test_sim = root_dir_test_base+simulation_case
  
    # Create the dataset with a subdirectory condition
    dataset_test = PropellerDataset(root_dir_test_sim, subdir_condition=subdir_condition)
    inputs_test, outputs_test = dataset_test[0:]

    # Assuming your input tensor is named `input_tensor`
    input_tensor_test = inputs_test.squeeze(1)  # Remove the singleton dimension at index 1
    # print("Input shape:", input_tensor_test.shape)  # Should print: torch.Size([6, 145, 7])

    output_tensor_test = outputs_test.squeeze(1)
    # print("Output shape:",output_tensor_test.shape)

    inputs_test_reshaped = input_tensor_test.reshape(-1, input_size)

    test_inputs_normalized = input_scaler.transform(inputs_test_reshaped.reshape(-1, input_size)).reshape(input_tensor_test.shape)

    test_inputs_tensor = torch.tensor(test_inputs_normalized, dtype=torch.float32).to(device)

    # Make predictions using the trained model
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        predicted_outputs = model(test_inputs_tensor)

    # Convert the predictions back to numpy and inverse scale the outputs
    predicted_outputs = predicted_outputs.cpu().detach().numpy()  # Convert tensor to numpy array
    predicted_outputs_original_scale = output_scaler.inverse_transform(predicted_outputs.reshape(-1, output_size))

    # Reshape the predictions to match the original sequence structure if needed
    predicted_outputs_original_scale = predicted_outputs_original_scale.reshape(input_tensor_test.shape[0], input_tensor_test.shape[1], output_size)
    predicted_outputs_original_scale = predicted_outputs_original_scale[0]

    # print(predicted_outputs_original_scale.shape)

    # Model predicted values
    ct_test_NN = predicted_outputs_original_scale[:,0]
    cq_test_NN = predicted_outputs_original_scale[:,1]

    # Load timesteps, CT and CQ from FLOWUnsteady simualtions
    time_steps = dataset_test.get_variable('time')

    ct_test_flowuns = dataset_test.get_variable('CT')
    cq_test_flowuns = dataset_test.get_variable('CQ')

    fft_ct_fu_real, fft_ct_fu_imag = dataset_test.get_variable('fft_ct')
    fft_cq_fu_real, fft_cq_fu_imag = dataset_test.get_variable('fft_cq')

    # Compute MAPE and R² Score
    mape_ct = mape(ct_test_flowuns[0], ct_test_NN)
    mape_cq = mape(cq_test_flowuns[0], cq_test_NN)
    r2_ct = r2_score(ct_test_flowuns[0], ct_test_NN)
    r2_cq = r2_score(cq_test_flowuns[0], cq_test_NN)
    relative_l2_norm_ct = relative_l2_norm(ct_test_flowuns[0], ct_test_NN)
    relative_l2_norm_cq = relative_l2_norm(cq_test_flowuns[0], cq_test_NN)


    # Store results
    mape_ct_list.append(mape_ct)
    mape_cq_list.append(mape_cq)
    r2_ct_list.append(r2_ct)
    r2_cq_list.append(r2_cq)
    relative_l2_norm_ct_list.append(relative_l2_norm_ct)
    relative_l2_norm_cq_list.append(relative_l2_norm_cq)

    # Print metrics for this simulation case
    print(f"Simulation Case: {simulation_case}")
    print(f"  MAPE Ct: {mape_ct:.2f}%, MAPE Cq: {mape_cq:.2f}%")
    print(f"  R² Ct: {r2_ct:.4f}, R² Cq: {r2_cq:.4f}")
    print(f"  ε Ct: {relative_l2_norm_ct:.2f}%, ε Cq: {relative_l2_norm_cq:.2f}%")

    # Plot the results
    if PLOT_RESULTS:
        # plt.figure()
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 2, 1)
        plt.plot(ct_test_NN, label = 'ct_NN')
        plt.plot((ct_test_flowuns[0]), label = 'ct_flowuns')
        plt.xlabel('Time [s]')
        plt.ylabel('Thrust coefficient, $C_T$')
        plt.title(simulation_case)
        plt.legend()

        # plt.figure()
        plt.subplot(1, 2, 2)
        plt.plot(cq_test_NN, label = 'cq_NN')
        plt.plot(cq_test_flowuns[0], label = 'cq_flowuns')
        plt.xlabel('Time [s]')
        plt.ylabel('Torque coefficient, $C_Q$')
        plt.title(simulation_case)
        plt.legend()

# Compute average metrics across all test cases
print("\nOverall Metrics Across All Test Cases:")
print(f"  Avg MAPE Ct: {np.mean(mape_ct_list):.2f}%, Avg MAPE Cq: {np.mean(mape_cq_list):.2f}%")
print(f"  Avg R² Ct: {np.mean(r2_ct_list):.4f}, Avg R² Cq: {np.mean(r2_cq_list):.4f}")
print(f"  Avg ε Ct: {np.mean(relative_l2_norm_ct_list):.2f}%, Avg ε Cq: {np.mean(relative_l2_norm_cq_list):.2f}%")
